In [39]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm
from tqdm import trange
import jsonlines
from os import listdir
import itertools
import re
from collections import Counter
import json
import pymorphy2 as pm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import save_npz
from scipy.sparse import load_npz

In [2]:
ok_set = set()
with open("accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())
    
cat_id = {}
with open("cat_id.json", mode="r") as input:
    cat_id = json.loads(input.read())
    
id_cat ={}
with open("id_cat.json", mode="r") as input:
    id_cat = json.loads(input.read())

"categories = [d['categories'] for d in data]\ncategories = sorted(set(list(itertools.chain(*categories))))\n\ncategories_dict = {c: 0 for c in categories}\nfor d in data:\n    for c in d['categories']:\n        if categories_dict.get(c) is not None:\n            categories_dict[c] += 1\n            \nok = [c for c in categories if categories_dict[c] > 5 and categories_dict[c] < 5000]\nok_set = set(ok)\ncategories_dict = {d['id']: [c for c in d['categories'] if c in ok_set] for d in tqdm(data)}\nlen(ok)"

In [3]:
texts = {item['id']: item['text'] for item in jsonlines.open('normalized_texts.jl', 'r')}

In [4]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.98)
text_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])

In [42]:
save_npz("sources/text_tfidf", text_tfidf)

In [5]:
ids_tfidf = sorted([id for id in texts.keys()])
#svd = TruncatedSVD(n_components=500, random_state=27)
#X_tfidf = svd.fit_transform(text_tfidf)
X_tfidf = text_tfidf

In [14]:
X = X_tfidf
ids = ids_tfidf

kf = KFold(n_splits=20, shuffle=True, random_state=27)


train_index, test_index = 0, 0
for item in kf.split(X):
    train_index, test_index = item[0], item[1]
    break
    
print("TRAIN:", train_index, "TEST:", test_index)
    
X_train = X_tfidf[train_index]
ids_train = [ids[i] for i in train_index]

class_centroids = []
for i, category in enumerate(tqdm(ok)):
    clf_centroids = NearestCentroid()
    y_train = np.array([1 if category in categories_dict[id] else 0 for id in ids_train])
    if sum(y_train) > 0:
        clf_centroids.fit(X_train, y_train)
        class_centroids.append(csr_matrix(clf_centroids.centroids_[1]))
    else:
         print(category)   

TRAIN: [    0     1     2 ... 96791 96792 96793] TEST: [   56    71    93 ... 96715 96756 96789]


100%|██████████████████████████████████████████████████████████████████████████| 14697/14697 [1:27:11<00:00,  2.81it/s]


In [25]:
class_centroids[0].shape[1]

121303

In [32]:
_class_centroids = lil_matrix((len(class_centroids), class_centroids[0].shape[1]))
for i in range(len(class_centroids)):
    _class_centroids[i] = class_centroids[i]

In [41]:
save_npz("sources/sparce_centroids_nosvd", _class_centroids.tocsr())

In [5]:
import numpy as np

a = np.array([1, 2])
b = np.array([3, 4])
dict.fromkeys(a, b)

{1: array([3, 4]), 2: array([3, 4])}